In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install llama_index pyvis Ipython langchain pypdf langchain_community
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-langchain
!pip install langchain-huggingface
!pip install sentencepiece accelerate
!pip install -U bitsandbytes
!pip install peft
!pip install llama-index-readers-wikipedia wikipedia
!pip install gradio

In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import KnowledgeGraphIndex
from llama_index.core import Settings
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch
import gradio as gr

#from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from pyvis.network import Network

In [ ]:
from huggingface_hub import login
HF_TOKEN="YOUR_HF_TOKEN"
# Вставьте ваш токен (здесь указан временный токен)
login(HF_TOKEN, add_to_git_credential=True)

In [ ]:
def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        elif message.role == 'bot':
            prompt += f"<s>bot\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<s>system\n"):
        prompt = "<s>system\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<s>bot\n"
    return prompt

def completion_to_prompt(completion):
    return f"<s>system\n</s>\n<s>user\n{completion}</s>\n<s>bot\n"

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM

# Определяем параметры квантования, иначе модель не выполниться в колабе
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Задаем имя модели
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"

# Создание конфига, соответствующего методу PEFT (в нашем случае LoRA)
config = PeftConfig.from_pretrained(MODEL_NAME)

# Загружаем базовую модель, ее имя берем из конфига для LoRA
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,          # идентификатор модели
    quantization_config=quantization_config, # параметры квантования
    torch_dtype=torch.float16,               # тип данных
    device_map="auto"                        # автоматический выбор типа устройства
)

# Загружаем LoRA модель
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)

# Переводим модель в режим инференса
# Можно не переводить, но явное всегда лучше неявного
model.eval()

# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

In [ ]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

In [ ]:
llm = HuggingFaceLLM(
    model=model,             # модель
    model_name=MODEL_NAME,   # идентификатор модели
    tokenizer=tokenizer,     # токенизатор
    max_new_tokens=generation_config.max_new_tokens, # параметр необходимо использовать здесь, и не использовать в generate_kwargs, иначе ошибка двойного использования
    model_kwargs={"quantization_config": quantization_config}, # параметры квантования
    generate_kwargs = {   # параметры для инференса
      "bos_token_id": generation_config.bos_token_id, # токен начала последовательности
      "eos_token_id": generation_config.eos_token_id, # токен окончания последовательности
      "pad_token_id": generation_config.pad_token_id, # токен пакетной обработки (указывает, что последовательность ещё не завершена)
      "no_repeat_ngram_size": generation_config.no_repeat_ngram_size,
      "repetition_penalty": generation_config.repetition_penalty,
      "temperature": generation_config.temperature,
      "do_sample": True,
      "top_k": 50,
      "top_p": 0.95
    },
    messages_to_prompt=messages_to_prompt,     # функция для преобразования сообщений к внутреннему формату
    completion_to_prompt=completion_to_prompt, # функции для генерации текста
    device_map="auto",                         # автоматически определять устройство
)

In [ ]:
!wget https://storage.yandexcloud.net/pdfreader/PC.pdf

In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.readers.file import PDFReader
from langchain.embeddings import HuggingFaceEmbeddings

def create_index():
    # PDF Reader with `SimpleDirectoryReader`
    parser = PDFReader()
    file_extractor = {"PC.pdf": parser}
    docs = SimpleDirectoryReader("/content"
    ).load_data()

    embed_model = LangchainEmbedding(
        HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    )

    Settings.llm = llm
    Settings.embed_model = embed_model
    Settings.chunk_size = 512

    graph_store = SimpleGraphStore()
    storage_context = StorageContext.from_defaults(graph_store=graph_store)

    indexKG = KnowledgeGraphIndex.from_documents(
        documents=docs,
        max_triplets_per_chunk=3,
        show_progress=True,
        include_embeddings=True,
        storage_context=storage_context
    )

    return indexKG

indexKG = None

def set_index():
    global indexKG
    indexKG = create_index()
    return "Индекс успешно создан!"

In [ ]:
def query_response(query):
    global indexKG
    if indexKG is None:
        return "Индекс не создан. Пожалуйста, создайте индекс сначала."

    query_engine = indexKG.as_query_engine(include_text=True, verbose=True)

    message_template = f"""<s>system
    Отвечай в соответствии с Источником. Проверь, есть ли в Источнике упоминания о ключевых словах Вопроса.
    Если нет, то просто скажи: 'я не знаю'. Не придумывай! </s>
    <s>user
    Вопрос: {query}
    Источник:
    </s>
    """
    response = query_engine.query(message_template)
    return response.response

In [ ]:
with gr.Blocks() as interface:
    gr.Markdown("# Устройство компьютера")
    gr.Markdown("Введите ваш вопрос.")

    with gr.Row():
        parse_button = gr.Button("Распарсить данные")
        query_input = gr.Textbox(label="Введите ваш вопрос")
        query_button = gr.Button("Отправить запрос")
        output_text = gr.Textbox(label="Ответ")

    parse_button.click(set_index, outputs=output_text)
    query_button.click(query_response, inputs=query_input, outputs=output_text)

interface.launch()